# Proyecto 2

In [ ]:
# Juan: Bloque 1 — Importaciones

import tkinter as tk  # Juan: Tkinter base

from tkinter import ttk, filedialog, messagebox  # Juan: Widgets/diálogos

from PIL import Image, ImageTk  # Juan: IO/preview de imágenes

import math  # Juan: Rotación trigonométrica

from typing import List  # Juan: Tipado


In [ ]:
# Juan: Bloque 2 — Utilidades de tensor

Tensor = List[List[List[int]]]  # Juan: Tensor [h][w][3]

def image_to_tensor(img: Image.Image) -> Tensor:  # Juan: PIL - tensor

    img = img.convert("RGB")  # Juan: Aseguro RGB

    w, h = img.size  # Juan: Dimensiones

    px = img.load()  # Juan: Acceso a píxeles

    t: Tensor = []  # Juan: Inicializo tensor

    for y in range(h):  # Juan: Filas

        row = []  # Juan: Fila temporal

        for x in range(w):  # Juan: Columnas

            r,g,b = px[x,y]  # Juan: Leo píxel

            row.append([int(r),int(g),int(b)])  # Juan: Guardo triplete

        t.append(row)  # Juan: Agrego fila

    return t  # Juan: Devuelvo tensor

def tensor_to_image(t: Tensor) -> Image.Image:  # Juan: tensor → PIL

    h = len(t)  # Juan: Altura

    w = len(t[0]) if h else 0  # Juan: Anchura

    out = Image.new("RGB",(w,h))  # Juan: Imagen vacía

    px = out.load()  # Juan: Acceso escritura

    for y in range(h):  # Juan: Filas

        for x in range(w):  # Juan: Columnas

            r,g,b = t[y][x]  # Juan: Triplete

            px[x,y] = (int(r),int(g),int(b))  # Juan: Escribo

    return out  # Juan: Retorno


In [ ]:
# Juan: Bloque 3 — Transformaciones (listas y bucles puros)

def to_grayscale(t: Tensor) -> Tensor:  # Juan: Escala de grises

    h = len(t)  # Juan: Altura

    w = len(t[0]) if h else 0  # Juan: Anchura

    out: Tensor = []  # Juan: Resultado

    for y in range(h):  # Juan: Filas

        row = []  # Juan: Fila

        for x in range(w):  # Juan: Columnas

            r,g,b = t[y][x]  # Juan: Píxel

            gval = int(0.299*r + 0.587*g + 0.114*b)  # Juan: Luma perceptual

            row.append([gval,gval,gval])  # Juan: RGB iguales

        out.append(row)  # Juan: Agrego fila

    return out  # Juan: Retorno

def resize_percent(t: Tensor, percent: int) -> Tensor:  # Juan: Escalado vecino cercano

    percent = max(1, int(percent))  # Juan: Evito 0

    h = len(t)  # Juan: Altura

    w = len(t[0]) if h else 0  # Juan: Anchura

    nw = max(1, (w*percent)//100)  # Juan: Nuevo ancho

    nh = max(1, (h*percent)//100)  # Juan: Nueva altura

    out: Tensor = []  # Juan: Resultado

    for y in range(nh):  # Juan: Filas nuevas

        row = []  # Juan: Fila

        sy = int(y*h/nh)  # Juan: Y origen

        if sy>=h: sy=h-1  # Juan: Tope

        for x in range(nw):  # Juan: Columnas nuevas

            sx = int(x*w/nw)  # Juan: X origen

            if sx>=w: sx=w-1  # Juan: Tope

            row.append(t[sy][sx])  # Juan: Copio píxel

        out.append(row)  # Juan: Agrego fila

    return out  # Juan: Retorno

def flip_horizontal(t: Tensor) -> Tensor:  # Juan: Invertir H

    h = len(t)  # Juan: Altura

    w = len(t[0]) if h else 0  # Juan: Anchura

    out: Tensor = []  # Juan: Resultado

    for y in range(h):  # Juan: Filas

        row = []  # Juan: Fila

        for x in range(w):  # Juan: Columnas

            row.append(t[y][w-1-x])  # Juan: Espejo

        out.append(row)  # Juan: Agrego fila

    return out  # Juan: Retorno

def flip_vertical(t: Tensor) -> Tensor:  # Juan: Invertir V

    h = len(t)  # Juan: Altura

    out: Tensor = []  # Juan: Resultado

    for y in range(h):  # Juan: Filas

        out.append(t[h-1-y])  # Juan: Inserto invertido

    return out  # Juan: Retorno

def rotate_degrees(t: Tensor, degrees: int) -> Tensor:  # Juan: Rotación 0-180°

    h = len(t)  # Juan: Altura

    w = len(t[0]) if h else 0  # Juan: Anchura

    ang = max(0, min(180, int(degrees)))  # Juan: Clampeo a 0..180

    rad = math.radians(ang)  # Juan: Radianes

    cs = math.cos(rad)  # Juan: Coseno

    sn = math.sin(rad)  # Juan: Seno

    cx = (w-1)/2.0  # Juan: Centro X

    cy = (h-1)/2.0  # Juan: Centro Y

    out: Tensor = []  # Juan: Resultado

    for y in range(h):  # Juan: Y salida

        row = []  # Juan: Fila

        for x in range(w):  # Juan: X salida

            dx = x - cx  # Juan: Rel X

            dy = y - cy  # Juan: Rel Y

            sx = int(round( dx*cs + dy*sn + cx ))  # Juan: X origen

            sy = int(round(-dx*sn + dy*cs + cy ))  # Juan: Y origen

            if 0<=sx<w and 0<=sy<h:  # Juan: Dentro

                row.append(t[sy][sx])  # Juan: Copio

            else:  # Juan: Fuera

                row.append([0,0,0])  # Juan: Negro

        out.append(row)  # Juan: Agrego fila

    return out  # Juan: Retorno


In [ ]:
# Juan: Bloque 4 — Aplicación Tkinter (UI rosa)

class App:  # Juan: Aplicación principal

    def __init__(self, root):  # Juan: Constructor

        self.root = root  # Juan: Guardar raíz

        self.root.title("Procesador de Imágenes")  # Juan: Título

        self.root.geometry("1120x660")  # Juan: Tamaño

        self.root.configure(bg="#160b14")  # Juan: Fondo base

        style = ttk.Style()  # Juan: Estilo ttk

        style.theme_use("clam")  # Juan: Tema

        style.configure("TFrame", background="#160b14")  # Juan: Fondo frames

        style.configure("TLabel", background="#160b14", foreground="#fde2f3", font=("Inter",10))  # Juan: Etiquetas

        style.configure("TButton", padding=8, relief="flat", background="#ec4899", foreground="#160b14", font=("Inter",10,"bold"))  # Juan: Botones

        style.map("TButton", background=[("active","#f472b6")])  # Juan: Hover

        style.configure("Pink.TLabelframe", background="#1a0f17", foreground="#f9a8d4", bordercolor="#be185d", relief="solid")  # Juan: Marco rosa

        style.configure("Pink.TLabelframe.Label", background="#1a0f17", foreground="#f9a8d4")  # Juan: Texto marco

        self.src_img = None  # Juan: PIL original

        self.src_tensor = None  # Juan: Tensor original

        self.out_img = None  # Juan: PIL resultado

        main = ttk.Frame(self.root)  # Juan: Contenedor

        main.pack(fill="both", expand=True, padx=12, pady=12)  # Juan: Márgenes


        # Juan: Panel izquierdo (controles)

        left = ttk.Labelframe(main, text="Controles", style="Pink.TLabelframe")  # Juan: Marco con título

        left.pack(side="left", fill="y", padx=(0,12))  # Juan: Empaquetado


        btn_open = ttk.Button(left, text="Abrir JPG/PNG", command=self.open_image)  # Juan: Botón abrir

        btn_open.pack(fill="x", pady=(8,8))  # Juan: Ubicación


        self.var_gray = tk.BooleanVar(value=False)  # Juan: Estado grises

        self.var_flip_h = tk.BooleanVar(value=False)  # Juan: Estado flip H

        self.var_flip_v = tk.BooleanVar(value=False)  # Juan: Estado flip V

        ttk.Checkbutton(left, text="Escala de grises", variable=self.var_gray).pack(anchor="w")  # Juan: Checkbox

        ttk.Checkbutton(left, text="Invertir Horizontal", variable=self.var_flip_h).pack(anchor="w")  # Juan: Checkbox

        ttk.Checkbutton(left, text="Invertir Vertical", variable=self.var_flip_v).pack(anchor="w")  # Juan: Checkbox


        ttk.Label(left, text="Redimensionar (%)").pack(anchor="w", pady=(10,0))  # Juan: Texto

        self.var_scale = tk.IntVar(value=100)  # Juan: Porcentaje

        self.scale = ttk.Scale(left, from_=10, to=400, variable=self.var_scale, orient="horizontal")  # Juan: Slider

        self.scale.pack(fill="x")  # Juan: Ubicación


        ttk.Label(left, text="Rotar (0-180°)").pack(anchor="w", pady=(10,0))  # Juan: Texto

        self.var_angle = tk.IntVar(value=0)  # Juan: Ángulo

        self.angle = ttk.Scale(left, from_=0, to=180, variable=self.var_angle, orient="horizontal")  # Juan: Slider

        self.angle.pack(fill="x")  # Juan: Ubicación


        ttk.Label(left, text="Orden: Grises → Redimensionar → Flip H → Flip V → Rotar").pack(anchor="w", pady=(10,8))  # Juan: Nota


        ttk.Button(left, text="Aplicar Transformaciones", command=self.process).pack(fill="x")  # Juan: Botón aplicar

        ttk.Button(left, text="Guardar Resultado", command=self.save).pack(fill="x", pady=(6,8))  # Juan: Botón guardar


        # Juan: Panel derecho (vistas)

        right = ttk.Labelframe(main, text="Vista y Descarga", style="Pink.TLabelframe")  # Juan: Marco

        right.pack(side="right", fill="both", expand=True)  # Juan: Empaquetado


        preview = ttk.Frame(right)  # Juan: Contenedor de imágenes

        preview.pack(fill="both", expand=True, padx=8, pady=8)  # Juan: Márgenes


        self.lbl_in = ttk.Label(preview, text="Original", anchor="center")  # Juan: Etiqueta original

        self.lbl_in.pack(side="left", fill="both", expand=True, padx=(0,6))  # Juan: Ubicación

        self.lbl_out = ttk.Label(preview, text="Resultado", anchor="center")  # Juan: Etiqueta resultado

        self.lbl_out.pack(side="right", fill="both", expand=True, padx=(6,0))  # Juan: Ubicación


    def open_image(self):  # Juan: Abrir archivo

        path = filedialog.askopenfilename(title="Selecciona imagen", filetypes=[("Imágenes","*.png;*.jpg;*.jpeg")])  # Juan: Diálogo

        if not path:  # Juan: Cancelado

            return  # Juan: Salgo

        try:  # Juan: Manejo de errores

            img = Image.open(path)  # Juan: Abro

            self.src_img = img  # Juan: Guardo

            self.src_tensor = image_to_tensor(img)  # Juan: A tensor

            self.show_image(self.lbl_in, img)  # Juan: Muestro

            self.lbl_out.config(text="Resultado")  # Juan: Reset

        except Exception as e:  # Juan: Error

            messagebox.showerror("Error", f"No se pudo abrir la imagen:\n{e}")  # Juan: Alerta



    def show_image(self, label, pil_img):  # Juan: Renderizar en label

        max_w, max_h = 520, 520  # Juan: Límites

        w, h = pil_img.size  # Juan: Tamaño

        scale = min(max_w/w, max_h/h, 1.0)  # Juan: Factor

        if scale < 1.0:  # Juan: Si excede

            pil_img = pil_img.resize((int(w*scale), int(h*scale)))  # Juan: Reduzco preview

        tk_img = ImageTk.PhotoImage(pil_img)  # Juan: TK image

        label.img_ref = tk_img  # Juan: Evita GC

        label.configure(image=tk_img)  # Juan: Coloca imagen


    def process(self):  # Juan: Ejecutar pipeline

        if self.src_tensor is None:  # Juan: Sin imagen

            messagebox.showinfo("Atención","Primero abre una imagen.")  # Juan: Aviso

            return  # Juan: Salgo

        t = self.src_tensor  # Juan: Base

        if self.var_gray.get():  # Juan: Grises

            t = to_grayscale(t)  # Juan: Aplico

        percent = int(self.var_scale.get())  # Juan: Porcentaje

        if percent != 100:  # Juan: Cambio tamaño

            t = resize_percent(t, percent)  # Juan: Aplico

        if self.var_flip_h.get():  # Juan: Flip H

            t = flip_horizontal(t)  # Juan: Aplico

        if self.var_flip_v.get():  # Juan: Flip V

            t = flip_vertical(t)  # Juan: Aplico

        angle = int(self.var_angle.get())  # Juan: Ángulo

        t = rotate_degrees(t, angle)  # Juan: Rotación (clamp interno 0..180)

        self.out_img = tensor_to_image(t)  # Juan: tensor → PIL

        self.show_image(self.lbl_out, self.out_img)  # Juan: Mostrar


    def save(self):  # Juan: Guardar resultado

        if self.out_img is None:  # Juan: Sin resultado

            messagebox.showinfo("Atención","No hay resultado para guardar.")  # Juan: Aviso

            return  # Juan: Salgo
        path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG","*.png"),("JPEG","*.jpg;*.jpeg")])  # Juan: Diálogo

        if not path:  # Juan: Cancelado

            return  # Juan: Salgo

        try:  # Juan: Intento guardar

            self.out_img.save(path)  # Juan: Guardado

            messagebox.showinfo("Éxito", f"Imagen guardada en:\n{path}")  # Juan: Confirmación

        except Exception as e:  # Juan: Error de IO

            messagebox.showerror("Error", f"No se pudo guardar:\n{e}")  # Juan: Alerta


In [ ]:
# Juan: Bloque 5 — Punto de entrada

if __name__ == "__main__":  # Juan: Punto de entrada

    root = tk.Tk()  # Juan: Ventana

    app = App(root)  # Juan: Instancia

    root.mainloop()  # Juan: Bucle
